In [2]:
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [20]:
#Import 
#preds = pd.read_csv('../data/tables/FlorenceThirdLabels20180917a.csv')
#preds = pd.read_csv('../data/tables/MichaelThirdLabels20181011a.csv')
preds = pd.read_csv('../data/tables/IsaiasThirdLabels20200804a.csv')

#raw images:
#aerial_folder = '../data/pics/raw/Florence_20180917a_jpgs/'
#aerial_folder = '../data/pics/raw/Michael_20181011a_jpgs/'
aerial_folder = '../data/pics/raw/Isaias_20200804a_jpgs/'

#save path
#ALPATH = '../data/pics/modeltoLabeler/roundThree/FlorenceWash/'
#ALPATH = '../data/pics/modeltoLabeler/roundThree/MichaelWash/'
ALPATH = '../data/pics/modeltoLabeler/roundThree/IsaiasWash/'

In [21]:
preds.head()

,Unnamed: 0,file,storm_id,archive,image,size,date,lr_lon,ul_lat,ll_lat,...,geom_checksum,ur_lat,lr_lat,ur_lon,prediction,prediction2,washover,labeled,pred_certainty,pred_certainty2
0,0,20200804a_jpgs/jpgs/S21776978.jpg,isaias,20200804a_jpgs,S21776978.jpg,6705271,2020/08/04,-79.889175,32.857442,32.854921,...,16b31b1f3d81d34022cc28f76a5cfb90,32.875906,32.866516,-79.904018,0.006467,0.108500,NaN,NaN,0.493533,0.391500
1,1,20200804a_jpgs/jpgs/S21766393.jpg,isaias,20200804a_jpgs,S21766393.jpg,6259383,2020/08/04,-75.959077,35.095388,35.086077,...,088ab652cbd4ed8ee48c712d7d0b3e20,35.107112,35.093768,-75.966710,0.753434,0.969360,NaN,NaN,0.253434,0.469360
2,2,20200804a_jpgs/jpgs/S21767012.jpg,isaias,20200804a_jpgs,S21767012.jpg,6009380,2020/08/04,-76.294318,34.863638,34.853882,...,db6d72612621b343057cea872f07e8e5,34.875547,34.861663,-76.302168,0.786389,0.947221,NaN,NaN,0.286389,0.447221
3,3,20200804a_jpgs/jpgs/S21775298.jpg,isaias,20200804a_jpgs,S21775298.jpg,5281908,2020/08/04,-79.351721,33.014702,33.004522,...,87f100dcdc56c94d80010dccbd82492c,33.019752,33.008178,-79.353651,0.641310,0.437554,NaN,NaN,0.141310,0.062446
4,4,20200804a_jpgs/jpgs/S21770086.jpg,isaias,20200804a_jpgs,S21770086.jpg,7279868,2020/08/04,-77.485597,34.457228,34.446724,...,c575a5c471b8be3efbb7fd4013a35c37,34.467243,34.453305,-77.491345,0.701684,0.940632,NaN,NaN,0.201684,0.440632


In [22]:
#make new column certainty of prediction, from 0 to 0.5... 0 is smallest certainty
preds["washImages"] = abs(preds["prediction2"].astype(float) - 0.75)

#select top 100 each that have not been labeled
uncertain_preds = preds[preds.labeled.isna()].nsmallest(100, 'washImages')

maxval = max(uncertain_preds['washImages'])

#add labeled = 3 for all the images that will be labeled
preds.loc[(preds['pred_certainty2'] <= maxval) & (preds['labeled'].isna()), 'labeled'] = 3

#preds.head()
uncertain_preds.head()

,Unnamed: 0,file,storm_id,archive,image,size,date,lr_lon,ul_lat,ll_lat,...,ur_lat,lr_lat,ur_lon,prediction,prediction2,washover,labeled,pred_certainty,pred_certainty2,washImages
610,610,20200804a_jpgs/jpgs/S21770389.jpg,isaias,20200804a_jpgs,S21770389.jpg,6880802,2020/08/04,-77.660543,34.334279,34.323801,...,34.346305,34.331207,-77.667756,1.824993e-02,0.750034,NaN,NaN,0.481750,0.250034,0.000034
2479,2479,20200804a_jpgs/jpgs/P21771635.jpg,isaias,20200804a_jpgs,P21771635.jpg,9776975,2020/08/04,-77.935234,34.022000,34.028565,...,34.043288,34.041278,-77.955455,7.807829e-10,0.749739,NaN,NaN,0.500000,0.249739,0.000261
2175,2175,20200804a_jpgs/jpgs/S21775147.jpg,isaias,20200804a_jpgs,S21775147.jpg,5908483,2020/08/04,-79.296601,33.102155,33.090454,...,33.111141,33.096124,-79.300777,2.394940e-01,0.749257,NaN,NaN,0.260506,0.249257,0.000743
2394,2394,20200804a_jpgs/jpgs/P21767963.jpg,isaias,20200804a_jpgs,P21767963.jpg,6553019,2020/08/04,-76.710259,34.686832,34.703878,...,34.689379,34.705658,-76.718681,9.514687e-01,0.750829,NaN,NaN,0.451469,0.250829,0.000829
2272,2272,20200804a_jpgs/jpgs/S21774427.jpg,isaias,20200804a_jpgs,S21774427.jpg,7216829,2020/08/04,-79.048891,33.503085,33.494075,...,33.515264,33.501864,-79.056473,2.461020e-01,0.751013,NaN,NaN,0.253898,0.251013,0.001013


In [23]:
#add the path
uncertain_preds['Image_Path'] = aerial_folder + uncertain_preds['image'].astype(str)

#pull out the list from the df
Labeled_list= uncertain_preds['Image_Path'].tolist()

#move those on the list
for f in Labeled_list:
    shutil.copy(f, ALPATH)